In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/train.csv")
df

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [3]:
df['eeg_id'].unique()

array([1628180742, 2277392603,  722738444, ..., 1850739625, 1306668185,
        351917269])

In [4]:
df[df['eeg_id'] == 1628180742]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
5,1628180742,5,26.0,353733,5,26.0,2413091605,42516,Seizure,3,0,0,0,0,0
6,1628180742,6,30.0,353733,6,30.0,364593930,42516,Seizure,3,0,0,0,0,0
7,1628180742,7,36.0,353733,7,36.0,3811483573,42516,Seizure,3,0,0,0,0,0
8,1628180742,8,40.0,353733,8,40.0,3388718494,42516,Seizure,3,0,0,0,0,0


In [5]:
df['total_votes'] = df['seizure_vote']+df['lpd_vote']+df['gpd_vote']+df['lrda_vote']+df['grda_vote']+df['other_vote']

In [6]:
df['seizure_vote']= df['seizure_vote']/df['total_votes']  
df['lpd_vote']= df['lpd_vote']/df['total_votes'] 
df['gpd_vote']= df['gpd_vote']/df['total_votes'] 
df['lrda_vote']= df['lrda_vote']/df['total_votes'] 
df['grda_vote']= df['grda_vote']/df['total_votes'] 
df['other_vote']= df['other_vote']/df['total_votes'] 

In [7]:
df['eeg_label_end'] = df['eeg_label_offset_seconds'] + 50

In [8]:
df_grouped = df.groupby(["eeg_id", 'seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote'])['eeg_label_offset_seconds'].apply(list).reset_index()

In [24]:
def windows(offsets):
    janelas = []
    inicio = offsets[0]
    fim = inicio + 50

    for i in offsets:
        if i <= fim:
            fim = i + 50
        if fim < i:
            janelas.append(f'{int(inicio)}-{int(fim)}')
            inicio = i
            fim = inicio + 50            

    janelas.append(f'{inicio}-{fim}')
    return janelas
            

In [25]:
df_grouped['janelas'] = df_grouped['eeg_label_offset_seconds'].apply(windows)

In [26]:
pd.options.display.max_colwidth = None

In [27]:
df_grouped[df_grouped['janelas'].apply(len) > 1][['eeg_label_offset_seconds','janelas']]

,eeg_label_offset_seconds,janelas
79,"[0.0, 2.0, 16.0, 36.0, 54.0, 58.0, 62.0, 86.0, 92.0, 94.0, 100.0, 106.0, 120.0, 190.0, 192.0, 198.0, 200.0, 204.0, 206.0, 212.0, 220.0, 232.0, 254.0, 256.0, 258.0, 264.0, 266.0, 272.0, 274.0]","[0-170, 190.0-324.0]"
135,"[0.0, 24.0, 80.0]","[0-74, 80.0-130.0]"
200,"[0.0, 98.0]","[0-50, 98.0-148.0]"
226,"[18.0, 86.0]","[18-68, 86.0-136.0]"
227,"[0.0, 54.0]","[0-50, 54.0-104.0]"
...,...,...
19257,"[0.0, 180.0]","[0-50, 180.0-230.0]"
19509,"[0.0, 2.0, 6.0, 14.0, 22.0, 30.0, 44.0, 132.0, 150.0, 158.0, 170.0, 172.0, 174.0, 176.0, 182.0, 186.0, 188.0, 190.0, 194.0, 200.0, 202.0]","[0-94, 132.0-252.0]"
19524,"[140.0, 240.0, 328.0, 350.0]","[140-190, 240-290, 328.0-400.0]"
19589,"[0.0, 4.0, 10.0, 20.0, 72.0, 78.0, 84.0, 86.0, 88.0, 146.0, 148.0, 150.0, 160.0, 176.0, 182.0, 200.0, 202.0, 206.0, 214.0, 226.0, 232.0, 238.0, 240.0, 256.0]","[0-70, 72-138, 146.0-306.0]"


In [30]:
df_grouped['janelas']=df_grouped['janelas'].apply(lambda x: ','.join(x))

In [33]:
df_grouped.head()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,eeg_label_offset_seconds,janelas
0,568657,0.0,0.000000,0.25,0.000000,0.166667,0.583333,"[0.0, 6.0, 12.0, 16.0]",0.0-66.0
1,582999,0.0,0.857143,0.00,0.071429,0.000000,0.071429,"[0.0, 4.0, 10.0, 14.0, 16.0, 18.0, 20.0, 22.0, 24.0, 26.0, 38.0]",0.0-88.0
2,642382,0.0,0.000000,0.00,0.000000,0.000000,1.000000,"[0.0, 24.0]",0.0-74.0
3,751790,0.0,0.000000,1.00,0.000000,0.000000,0.000000,[0.0],0.0-50.0
4,778705,0.0,0.000000,0.00,0.000000,0.000000,1.000000,[0.0],0.0-50.0


In [34]:
df_grouped.to_csv('data/scores.csv')